In [1]:
!pip install transformers

'pip' is not recognized as an internal or external command,
operable program or batch file.


In [2]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import cross_val_score
import torch
import transformers as ppb
import warnings

warnings.filterwarnings('ignore')

In [3]:
#For logistic regression
from sklearn.linear_model import LogisticRegression

In [4]:
#For naive bayes
from sklearn.naive_bayes import GaussianNB
from sklearn.naive_bayes import MultinomialNB
from sklearn.naive_bayes import CategoricalNB

In [5]:
#for linear svc
from sklearn import svm

In [6]:
df = pd.read_csv("FormattedTrainingDataset.csv", delimiter=',', header=None)

In [7]:
#DistilBERT
model_class, tokenizer_class, pretrained_weights = (ppb.DistilBertModel, ppb.DistilBertTokenizer, 'distilbert-base-uncased')
tokenizer = tokenizer_class.from_pretrained(pretrained_weights)
model = model_class.from_pretrained(pretrained_weights)

In [8]:
###BERT
model_class, tokenizer_class, pretrained_weights = (ppb.BertModel, ppb.BertTokenizer, 'bert-base-uncased')
tokenizer = tokenizer_class.from_pretrained(pretrained_weights)

model = model_class.from_pretrained(pretrained_weights)

In [9]:
batch_1 = df[:3800]

In [10]:
batch_1[1].value_counts()

value     2182
policy     815
fact       786
Name: 1, dtype: int64

In [11]:
tokenized = batch_1[0].apply((lambda x: tokenizer.encode(x, add_special_tokens=True)))

In [12]:
max_len = 0
for i in tokenized.values:
  if len(i) > max_len:
    max_len = len(i)
padded = np.array([i + [0]*(max_len-len(i)) for i in tokenized.values])

In [14]:
np.array(padded).shape

(3783, 116)

In [16]:
attention_mask = np.where(padded != 0, 1, 0)
attention_mask.shape


(3783, 116)

In [17]:
input_ids = torch.LongTensor(padded)
attention_mask = torch.tensor(attention_mask)

with torch.no_grad():
  last_hidden_states = model(input_ids, attention_mask=attention_mask)

In [18]:
last_hidden_states[0][:,0,:].numpy()

array([[-0.2607667 , -0.20353088, -0.22524181, ...,  0.12441164,
         0.07126848,  0.35151252],
       [-0.34178323, -0.62609017, -1.1245483 , ...,  0.02805318,
        -0.38737717,  0.5007663 ],
       [-0.17111711, -0.05117545, -0.15776686, ..., -0.1635767 ,
         0.326662  ,  0.5927742 ],
       ...,
       [ 0.22074401, -0.14013086,  0.15528093, ..., -0.2823073 ,
         0.21121702,  0.48165104],
       [ 0.13924822,  0.26115525, -0.08441553, ..., -0.05328125,
         0.19245926,  0.47801524],
       [-0.13620552, -0.06756303, -0.10995404, ...,  0.11258655,
         0.02763032,  0.44490597]], dtype=float32)

In [19]:
features = last_hidden_states[0][:,0,:].numpy()

In [20]:
labels = batch_1[1]

In [47]:
from statistics import mean, stdev
from sklearn import preprocessing
from sklearn.model_selection import StratifiedKFold
from sklearn import linear_model
from sklearn import datasets

In [60]:
from sklearn.metrics import f1_score
scaler = preprocessing.MinMaxScaler()
features_scaled = scaler.fit_transform(features)
skf = StratifiedKFold(n_splits=10, shuffle=True, random_state=1)
lst_accu_stratified = []
lst_f1_stratified = []



In [66]:
lr = linear_model.LogisticRegression(C = 5.263252631578947)

for train_index, test_index in skf.split(features, labels):
    x_train_fold, x_test_fold = features_scaled[train_index], features_scaled[test_index]
    y_train_fold, y_test_fold = labels[train_index], labels[test_index]
    lr.fit(x_train_fold, y_train_fold)
    lst_accu_stratified.append(lr.score(x_test_fold, y_test_fold))
    lst_f1_stratified.append(f1_score(y_test_fold, lr.predict(x_test_fold), average='weighted'))
   
# Print the output.
print('List of possible accuracy:', lst_accu_stratified)
print('\nMaximum Accuracy That can be obtained from this model is:',
      max(lst_accu_stratified)*100, '%')
print('\nMinimum Accuracy:',
      min(lst_accu_stratified)*100, '%')
print('\nOverall Accuracy:',
      mean(lst_accu_stratified)*100, '%')
print('\nStandard Deviation is:', stdev(lst_accu_stratified))

print('List of possible f1 scores:', lst_f1_stratified)
print('\nMaximum f1 score That can be obtained from this model is:',
      max(lst_f1_stratified)*100, '%')
print('\nMinimum f1 score:',
      min(lst_f1_stratified)*100, '%')
print('\nOverall f1 score:',
      mean(lst_f1_stratified)*100, '%')
print('\nStandard Deviation is:', stdev(lst_f1_stratified))

List of possible accuracy: [0.7783641160949868, 0.8021108179419525, 0.7994722955145118, 0.7566137566137566, 0.8148148148148148, 0.783068783068783, 0.7724867724867724, 0.7883597883597884, 0.791005291005291, 0.8095238095238095, 0.7783641160949868, 0.7783641160949868, 0.8021108179419525, 0.7994722955145118, 0.7566137566137566, 0.8148148148148148, 0.783068783068783, 0.7724867724867724, 0.7883597883597884, 0.791005291005291, 0.8095238095238095]

Maximum Accuracy That can be obtained from this model is: 81.48148148148148 %

Minimum Accuracy: 75.66137566137566 %

Overall Accuracy: 78.90478384259009 %

Standard Deviation is: 0.0170294415434534
List of possible f1 scores: [0.7734101985594833, 0.8021436682404329, 0.799791866525807, 0.7491399564228697, 0.8141706569625683, 0.7783605278348102, 0.7697675125107143, 0.7806741684445652, 0.791801023051023, 0.8035661841829242]

Maximum f1 score That can be obtained from this model is: 81.41706569625683 %

Minimum f1 score: 74.91399564228696 %

Overall f1

In [ ]:
lr_predicted = lr.predict(test_features)
print(classification_report(test_labels, lr_predicted))

In [21]:
train_features, test_features, train_labels, test_labels = train_test_split(features, labels)

In [39]:
#linearSVC score
parameters = {'C': np.linspace(0.0001, 100, 20)}
grid_search = GridSearchCV(svm.LinearSVC(), parameters)
grid_search.fit(train_index)

print('best parameters: ', grid_search.best_params_)
print('best scrores: ', grid_search.best_score_)



NameError: name 'labels_train' is not defined

In [ ]:
svc_clf = svm.LinearSVC(C=10.526405263157894)
svc_clf.fit(train_features, train_labels)

svc_predicted = svc_clf.predict(test_features)

In [38]:
#Logistic regression score
parameters = {'C': np.linspace(0.0001, 100, 20)}
grid_search = GridSearchCV(LogisticRegression(), parameters)
grid_search.fit(features_train, labels_train)

print('best parameters: ', grid_search.best_params_)
print('best scrores: ', grid_search.best_score_)

NameError: name 'features_train' is not defined

In [24]:
lr_clf = LogisticRegression(C=5.263252631578947)
lr_clf.fit(train_features, train_labels)


lr_predicted = lr_clf.predict(test_features)

In [ ]:
nb_clf = GaussianNB()
nb_clf.fit(train_features, train_labels)
nb_predicted = nb_clf.predict(test_features)
nb_clf.score(test_features, test_labels)

0.636

In [ ]:
from sklearn.metrics import classification_report

print("Naive Bayes Score:")
print(classification_report(test_labels, nb_predicted))
print("Linear regression Score:")
print(classification_report(test_labels, lr_predicted))
print("LinearSVC Score:")
print(classification_report(test_labels, svc_predicted))


Naive Bayes Score:
              precision    recall  f1-score   support

        fact       0.43      0.52      0.47       156
      policy       0.61      0.65      0.63       156
       value       0.75      0.67      0.71       438

    accuracy                           0.64       750
   macro avg       0.59      0.61      0.60       750
weighted avg       0.65      0.64      0.64       750

Linear regression Score:
              precision    recall  f1-score   support

        fact       0.60      0.69      0.64       156
      policy       0.86      0.86      0.86       156
       value       0.86      0.82      0.84       438

    accuracy                           0.80       750
   macro avg       0.77      0.79      0.78       750
weighted avg       0.81      0.80      0.80       750

LinearSVC Score:
              precision    recall  f1-score   support

        fact       0.50      0.62      0.56       156
      policy       0.73      0.88      0.80       156
       value  

In [25]:
from sklearn.metrics import confusion_matrix

confusion_matrix(test_labels, lr_predicted)

array([[108,   9,  69],
       [  5, 171,  27],
       [ 93,  23, 441]], dtype=int64)

In [ ]:
from sklearn.dummy import DummyClassifier
clf = DummyClassifier()

scores = cross_val_score(clf, train_features, train_labels)
print("Dummy classifier score: %0.3f (+/- %0.2f)" % (scores.mean(), scores.std() * 2))

Dummy classifier score: 0.356 (+/- 0.05)
